In [29]:
#association

## import module

In [4]:
import os,glob,gzip


## setting Data Location

### essential set : 
1. wDir
2. vcfDir

### input data in (wDir + INPUTs/)
1. ped file
2. chunk index file

In [2]:
#dir
wDir = "/DATA/smkim/KCHIP_130K/Asso/"
vcfDir = "/LaCie/ghyoon/OAS/"
outdir =  wDir + "RESULTs/"
inDir = wDir + "INPUTs/"
scriptDir = wDir +"SCRIPTs/"
versions = ["V1","V2"]
concepts = []
phenotypes = []

## Fucntion related File IO

In [3]:
def fileRead(fileIn):
    print("Read File : "+fileIn)
    if not os.path.exists(fileIn):
        print("Error! File does not exist in path :" + fileIn)
        return 0
    fileIO = open(fileIn,'r')
    InData = [f.replace("\n","").replace("\r","") for f in fileIO]
    
    fileIO.close()
    return InData
def make_dir(path):
    print("Make Dir path : "+ path)
    if not os.path.exists(path):
        os.makedirs(path)
        print("Make Dir Success")
        return path
    print("Already exist Dir! pass this step!")
    return path  

def gzRead(fileIn):
    print("Read gz File : "+fileIn)
    if not os.path.exists(fileIn):
        print("Error! File does not exist in path :" + fileIn)
        return 0
    fileIO = gzip.open(fileIn,'r')
    inData = [f.replace("\n","").replace("\r","") for f in fileIO]
    return inData


## Print option to select when using Switch : failure....TT

In [4]:
def print_option():
    print("please, Select option you want.")
    print("1. Make Shell scipt")
    print("2. Merge result file ")
    print("3. Merge result file(using pandas) ")
    print("4. Nothing")
    value = int(input("Input : "))
    return value

## main process function for association (asso, merge)
### 3 options 
1. asso
2. merge epacts 
3. merge epacts(pandas)

In [5]:
def make_assoSh(pedIn,chunksSplit):

    print("Make_assosh...")
    for phenotype in phenotypes:
        shDir = make_dir(scriptDir + "aasoQT_"+phenotype+"/")
        traitDir = make_dir(outdir + "assoRESULTs/"+phenotype+"/")
        for version in versions:
            for region_list in chunksSplit:
                
                region = region_list[0].replace("chr","")+":"+region_list[1]+"-"+region_list[2]
                vcfData = vcfDir + region_list[0]+ "_"+region_list[1] +"_"+region_list[2]+"_"+version+"_annoINFO_filINFO0.8.vcf.gz"
                runType = "_q.linear_" + phenotype+"_"+concept
                
                assoOut = vcfData.replace(vcfDir,traitDir).replace("_annoINFO_filINFO0.8.vcf.gz",runType)
                shOut = vcfData.replace(vcfDir,shDir).replace("_annoINFO_filINFO0.8.vcf.gz",runType + "_assoEPACTs.sh")

                with open(shOut, 'w') as shWrite:
                    shWrite.write("epacts-single --vcf "+ vcfData + " --ped " + pedIn +
                                  " --pheno "+ phenotype + " --test q.linear --run 8 --field DS --min-mac 5 -min-callrate 0.95 -no-plot"+
                                  " --region " + region +
                                  " --missing NA --out "+ assoOut + " \n")


In [7]:
def merge_asso(chunks):
    print("merge_ass......")
    mergeDir = make_dir(outDir + "assoMERGEs/")
    for phenotype in phenotypes:
        traitDir = make_dir(outdir + "assoRESULTs/"+phenotype+"/")
        mergeOut = make_dir(mergeDir + phenotype+"/")
        for version in versions:
            mergeOut = mergeOut + phenotype+"_"+version+"_mergeData.txt"
            with open(mergeOut,'w') as mergeWrite:
                for i,chunk in enumerate(chunks):
                    epactsIn = traitDir + chunk + "_" +version + "_q.linear_"+phenotype + ".epacts.gz"
                    if i != 0 and os.path.isfile(epactsIn):
                        mergeData = gzRead(epactsIn)
                        mergeWrite.write("\n".join(mergeData[1:])+"\n")
                    if i == 0 :
                        mergeData = gzRead(epactsIn)
                        mergeWrite.write("\n".join(mergeData[0:])+"\n")

                    if os.path.isfile(epactsIn):
                        mergeData = gzRead(epactsIn)
                        mergeWrite.write("\n".join(mergeData[1:])+"\n")
                    else:
                        os.system("rm -rf " +mergeOut)
                        break

            

In [6]:
## pandas가 있을 경우
import pandas as pd
def merge_asso_pd(chunks):
    print("merge_asso using pandas......")
    mergeDir = make_dir(outDir + "assoMERGEs/")
    for phenotype in phenotypes:
        traitDir = make_dir(outdir + "assoRESULTs/"+phenotype+"/")
        mergeOut = make_dir(mergeDir + phenotype + "/" )
        for version in versions:
            df = pd.DataFrame()          
            mergeOut = mergeOut + phenotype+"_"+version+"_mergeData.txt"
            for chunk in chunks:
                epactsIn = traitDir + chunk + "_" +version+"_q.linear_"+phenotype+".epacts.gz" 
                tmp = pd.read_csv(epactsIn,compression = 'gzip',delim_whitespace = True, error_bad_lines = False)
                df = pd.concat(df,tmp,axis=0,ignore_index=True)
            df.to_csv(mergeOut, index = False, sep = '\t',quotechar = "\"")
            print("TO_csv path : " + mergeOut)


# Main function

In [9]:
def main():
    Chunk = inDir+"imputation.IMPUTE4.POS.50K_20181114_Final.txt"
    Ori_chunksfile = [f.split("\t") for f in fileRead(Chunk)]
    chunks = [r[1] for r in Ori_chunksfile[1:]]
    chunksSplit = [r.split("_") for r in chunks]
    
    pedfile = "KCHIP130K_transformation_hematological_20190903.ped"
    pedIn = inDir + pedfile
    
    value = 0
    while(value !=4 ):
        value = print_option()
        if value == 1 :
            make_assoSh(pedIn,chunksSplit)
        elif value == 2 :
            merge_asso(chunks)
        elif value == 3 :
            merge_asso_pd(chunks)
        elif value == 4 :
            print("See you...")
            return
    
    
    #shDir = make_dir(scriptDir + "assoQT_"+phenotype +"/")
    #mergeDir = make_dir(outDir + "assoMERGEs/")
    #traitDir = outDir+"assoRESULTs/"+version+"_"+concept+"_"+phenotype + "/"

## main()

## exmaple codes and Data to guide

In [16]:
#df = pd.read_csv('sample.tar.gz', compression='gzip', header=0, sep=' ', quotechar='"', error_bad_lines=False)
df = pd.read_csv("C:/Users/user/Downloads/All_2017_BMI_BBJ_autosome.txt.gz",header = 0,sep='\t',error_bad_lines = False)

In [17]:
df.head()

,SNP,CHR,POS,REF,ALT,Frq,Rsq,BETA,SE,P
0,chr10_103577643,10,103577643,C,T,0.0302,0.995,-0.006703,0.010480,0.522500
1,chr10_104678841,10,104678841,G,A,0.0198,0.759,-0.044190,0.014610,0.002493
2,chr10_106151962,10,106151962,C,G,0.1394,0.820,0.003471,0.005649,0.538900
3,chr10_111967649,10,111967649,C,G,0.0295,0.928,0.008321,0.010850,0.443100
4,chr10_119805686,10,119805686,T,G,0.0184,0.982,-0.005624,0.013230,0.670700


In [2]:
ped = pd.read_csv("C:/Users/user/Desktop/KCDC/association/KCHIP130K_transformation_hematological_20190903.ped",delim_whitespace = True)

In [10]:
ped.columns

Index(['FAM_ID', 'IND_ID', 'FAT_ID', 'MOT_ID', 'age', 'sex', 'cohort', 'wbc',
       'rbc', 'hb', 'hct', 'plat', 'mcv', 'mch', 'mchc', 'smk', 'WBC_res',
       'WBC_z', 'RBC_res', 'RBC_z', 'PLAT_res', 'PLAT_z', 'HB_res', 'HB_z',
       'HCT_res', 'HCT_z', 'MCV_res', 'MCV_z', 'MCH_res', 'MCH_z', 'MCHC_res',
       'MCHC_z'],
      dtype='object')

In [12]:
list(filter(lambda x : "_z" in x,ped.columns))

['WBC_z', 'RBC_z', 'PLAT_z', 'HB_z', 'HCT_z', 'MCV_z', 'MCH_z', 'MCHC_z']

In [17]:
[name for name in ped.columns if "_z" in name]

['WBC_z', 'RBC_z', 'PLAT_z', 'HB_z', 'HCT_z', 'MCV_z', 'MCH_z', 'MCHC_z']

In [18]:
Chunk = "c:/Users/user/Desktop/KCDC/BMI_association/imputation.IMPUTE4.POS.50K_20181114_Final.txt"
Ori_chunksfile = [f.split("\t") for f in fileRead(Chunk)]
print(Ori_chunksfile[0:5])
chunks = [r[1] for r in Ori_chunksfile[1:]]
print("\nchunks : ")
print(chunks[0:5])
chunksSplit = [r.split("_") for r in chunks]
print("\nchunksSplit : ")
print(chunksSplit[0:5])

Read File : c:/Users/user/Desktop/KCDC/BMI_association/imputation.IMPUTE4.POS.50K_20181114_Final.txt
[['REFPANEL', 'IMPUTATION'], ['chr1_10178_5000000', 'chr1_10235_1891206'], ['chr1_10178_5000000', 'chr1_1891207_3156922'], ['chr1_10178_5000000', 'chr1_3156923_4581776'], ['chr1_10178_5000000', 'chr1_4581777_4999954']]

chunks : 
['chr1_10235_1891206', 'chr1_1891207_3156922', 'chr1_3156923_4581776', 'chr1_4581777_4999954', 'chr1_5000010_6381153']

chunksSplit : 
[['chr1', '10235', '1891206'], ['chr1', '1891207', '3156922'], ['chr1', '3156923', '4581776'], ['chr1', '4581777', '4999954'], ['chr1', '5000010', '6381153']]


In [7]:
while(print_option() !=3):
    print("while")
    

please, Select option you want.
1. Make Shell scipt
2. Merge result file
3. Nothing


Input :  1


while
please, Select option you want.
1. Make Shell scipt
2. Merge result file
3. Nothing


Input :  2


while
please, Select option you want.
1. Make Shell scipt
2. Merge result file
3. Nothing


Input :  3


In [20]:
name1 = pd.Series(["Kim","Lee","Choi","Son"])
age1 = pd.Series([10,20,19,29])
df1 = pd.DataFrame({"name" : name1,"age" : age1})
print("df1 : ")
print(df1)

name2 = pd.Series(["Lee","Park"])
age2 = pd.Series([20,99])
df2 = pd.DataFrame({"name":name2,"age":age2})
print("df2 : ")
print(df2)

print("Merge (concat)  : ")
merge_df = pd.concat([df1,df2],axis = 0,ignore_index=True)
print(merge_df)

merge_df = pd.merge(df1,df2,"outer")
print(merge_df)

df1 : 
   name  age
0   Kim   10
1   Lee   20
2  Choi   19
3   Son   29
df2 : 
   name  age
0   Lee   20
1  Park   99
Merge (concat)  : 
   name  age
0   Kim   10
1   Lee   20
2  Choi   19
3   Son   29
4   Lee   20
5  Park   99
   name  age
0   Kim   10
1   Lee   20
2  Choi   19
3   Son   29
4  Park   99


In [12]:
pd.merge

<function pandas.core.reshape.merge.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)>

In [13]:
pd.concat

<function pandas.core.reshape.concat.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=None, copy=True)>

<module 'pandas' from 'C:\\Users\\user\\Python\\Anaconda\\lib\\site-packages\\pandas\\__init__.py'>